## Import libraries

In [51]:
import geopandas as gpd
import numpy as np
import pandas as pd
import folium
from shapely.affinity import translate
from shapely.geometry import MultiPoint, Point
from shapely.ops import voronoi_diagram


## Import the dataset: Airbnbs in Amsterdam

In [49]:
df = pd.read_csv("listings.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2818,Quiet Garden View Room & Super Fast WiFi,3159,Daniel,NaN,Oostelijk Havengebied - Indische Buurt,52.365755,4.941419,Private room,59,3,248,2018-11-28,2.10,1,44
1,3209,"Quiet apt near center, great view",3806,Maartje,NaN,Westerpark,52.390225,4.873924,Entire home/apt,160,4,42,2018-08-29,1.03,1,47
2,20168,100%Centre-Studio 1 Private Floor/Bathroom,59484,Alex,NaN,Centrum-Oost,52.365087,4.893541,Entire home/apt,80,1,233,2018-11-30,2.18,2,198
3,25428,Lovely apt in City Centre (Jordaan),56142,Joan,NaN,Centrum-West,52.373114,4.883668,Entire home/apt,125,14,1,2018-01-21,0.09,2,141
4,27886,"Romantic, stylish B&B houseboat in canal district",97647,Flip,NaN,Centrum-West,52.386727,4.892078,Private room,150,2,171,2018-11-25,2.03,1,199


In [5]:
# to geodataframe and into a metric crs
gdf = (
    gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
        crs="EPSG:4326"
    )
    .to_crs("EPSG:3857")
)


points = gdf[:15].geometry
points.head()

0    POINT (550076.273 6866530.672)
1    POINT (542562.748 6870993.036)
2    POINT (544746.493 6866408.986)
3    POINT (543647.457 6867872.536)
4     POINT (544583.64 6870355.049)
Name: geometry, dtype: geometry

## Donut-masking

In [44]:
def donut_mask(geoms: gpd.GeoSeries, r_min: float, r_max: float, seed=42) -> gpd.GeoSeries:
    geoms = geoms.reset_index(drop=True)

    rng = np.random.default_rng(seed)
    angles = rng.uniform(0, 2*np.pi, len(geoms))
    radii  = rng.uniform(r_min, r_max, len(geoms))
    dx = radii * np.cos(angles)
    dy = radii * np.sin(angles)

    masked = [translate(g, xoff=float(dx[i]), yoff=float(dy[i])) for i, g in enumerate(geoms)]
    result = gpd.GeoSeries(masked, crs=geoms.crs)
    d = geoms.distance(result)
    print(d.describe())
    return result


## Set the radius
### You can play with the radius of the donut and have a look on how the results differ from each other

In [46]:
radius_min = 300
radius_max = 600

In [48]:
masked = donut_mask(points, radius_min, radius_max)

count     15.000000
mean     452.045092
std       94.477042
min      313.141130
25%      363.281614
50%      466.375436
75%      530.470686
max      591.209407
dtype: float64


## Plot the map
### Red = Original points
### Blue = Masked points

In [69]:
points_ll = points.to_crs("EPSG:4326")
masked_ll = masked.to_crs("EPSG:4326")

center = [points_ll.y.mean(), points_ll.x.mean()]
m = folium.Map(location=center, zoom_start=14)

fg_orig = folium.FeatureGroup(name="Original")
fg_mask = folium.FeatureGroup(name="Masked (Donut)")

for p in points_ll:
    folium.CircleMarker([p.y, p.x], radius=4, color="red", fill=True, fill_opacity=0.7).add_to(fg_orig)

for p in masked_ll:
    folium.CircleMarker([p
                         .y, p.x], radius=4, color="blue", fill=True, fill_opacity=0.7).add_to(fg_mask)

fg_orig.add_to(m)
fg_mask.add_to(m)
folium.LayerControl().add_to(m)

m


# Voronoi masking

### Define how many points you want for the voronoi diagrams. Try out with fewer and with a lot of points

In [52]:
points_count = 200

### Create the Voronoi diagrams

In [61]:
voronoi_points = gdf[15:15+points_count].geometry

In [62]:
envelope = points.unary_union.envelope.buffer(50)

voro = voronoi_diagram(MultiPoint(list(voronoi_points.values)), envelope=envelope)

cells = gpd.GeoDataFrame(
    {"cid": np.arange(len(voro.geoms))},
    geometry=list(voro.geoms),
    crs=points.crs
)

/tmp/ipykernel_1972016/1381577605.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  envelope = points.unary_union.envelope.buffer(50)


In [55]:
pts_gdf = gpd.GeoDataFrame(
    {"pid": np.arange(len(points))},
    geometry=points,
    crs=points.crs
)

joined = gpd.sjoin(pts_gdf, cells, predicate="within", how="left")


## Voronoi masking using the centroids

In [63]:
centroids = cells.set_index("cid").geometry.centroid
masked_voronoi = gpd.GeoSeries(joined["cid"].map(centroids).values, crs=points.crs)
joined["cid"].isna().sum()
points.distance(masked_voronoi).describe()

count      15.000000
mean      336.695103
std       296.358022
min        99.651295
25%       191.149572
50%       283.324902
75%       333.950983
max      1333.740694
dtype: float64

In [64]:
points_ll = points.to_crs(4326)
masked_ll = masked_voronoi.to_crs(4326)
cells_ll = cells.to_crs(4326)

center = [points_ll.y.mean(), points_ll.x.mean()]
m = folium.Map(location=center, zoom_start=14)

fg_orig = folium.FeatureGroup(name="Original")
fg_mask = folium.FeatureGroup(name="Masked (Donut)")

for p in points_ll:
    folium.CircleMarker([p.y, p.x], radius=4, color="red", fill=True, fill_opacity=0.7).add_to(fg_orig)

for p in masked_ll:
    folium.CircleMarker([p.y, p.x], radius=4, color="blue", fill=True, fill_opacity=0.7).add_to(fg_mask)

fg_orig.add_to(m)
fg_mask.add_to(m)


fg_cells = folium.FeatureGroup(name="Voronoi cells", show=True)

folium.GeoJson(
    data=cells_ll.to_json(),
    name="Voronoi cells",
    style_function=lambda feature: {
        "fillOpacity": 0.05,
        "weight": 1
    },
    tooltip=folium.GeoJsonTooltip(fields=["cid"], aliases=["Cell:"])
).add_to(fg_cells)

fg_cells.add_to(m)

folium.LayerControl().add_to(m)
m


## Voronoi masking with random points inside the polygon

In [65]:
def random_point_in_polygon(poly, rng, max_tries=10_000):
    minx, miny, maxx, maxy = poly.bounds
    for _ in range(max_tries):
        p = Point(rng.uniform(minx, maxx), rng.uniform(miny, maxy))
        if poly.contains(p):
            return p
    # Fallback (rare): return centroid if sampling fails
    return poly.centroid

In [67]:
rng = np.random.default_rng(42)
cell_geom = cells.set_index("cid").geometry

masked_voronoi_random = gpd.GeoSeries(
    [random_point_in_polygon(cell_geom.loc[cid], rng) for cid in joined["cid"]],
    crs=points.crs
)

points.distance(masked_voronoi_random).describe()

count      15.000000
mean      383.044739
std       463.549360
min        83.722425
25%       178.478055
50%       278.374963
75%       381.530928
max      1992.697273
dtype: float64

In [68]:
points_ll = points.to_crs(4326)
masked_ll = masked_voronoi_random.to_crs(4326)
cells_ll = cells.to_crs(4326)

center = [points_ll.y.mean(), points_ll.x.mean()]
m = folium.Map(location=center, zoom_start=14)

fg_orig = folium.FeatureGroup(name="Original")
fg_mask = folium.FeatureGroup(name="Masked (Donut)")

for p in points_ll:
    folium.CircleMarker([p.y, p.x], radius=4, color="red", fill=True, fill_opacity=0.7).add_to(fg_orig)

for p in masked_ll:
    folium.CircleMarker([p.y, p.x], radius=4, color="blue", fill=True, fill_opacity=0.7).add_to(fg_mask)

fg_orig.add_to(m)
fg_mask.add_to(m)


fg_cells = folium.FeatureGroup(name="Voronoi cells", show=True)

folium.GeoJson(
    data=cells_ll.to_json(),
    name="Voronoi cells",
    style_function=lambda feature: {
        "fillOpacity": 0.05,
        "weight": 1
    },
    tooltip=folium.GeoJsonTooltip(fields=["cid"], aliases=["Cell:"])
).add_to(fg_cells)

fg_cells.add_to(m)

folium.LayerControl().add_to(m)
m